In [1]:
# May need to install these....
# uncomment if unsure
!pip install wget
!pip install dtale

In [2]:
import pandas as pd
import numpy as np
import os as os

import dtale # data Frame visualization
import wget as wget

# Download the data

Download Current employment statistics from [US Bureau of Labor Statics website](https://www.bls.gov/)


Data Dictionary
???



In [3]:
# Check if file has ben downloaded if not download the file
# This could take a while first time downloading the files (over 5 mins)
# Does this qualify as web scraping? :-)
lnFileName = 'resources/ALLlnSeries.txt'
lnSeriesFileName = 'resources/lnSeries.txt'
lnIndustryFileName = 'resources/lnIndustries.txt'
if not os.path.exists(lnFileName) :
    url = "https://download.bls.gov/pub/time.series/ln/ln.data.1.AllData"
    wget.download(url,lnFileName)
    
    url = "https://download.bls.gov/pub/time.series/ln/ln.series"    
    wget.download(url,lnSeriesFileName)
    
    url = "https://download.bls.gov/pub/time.series/ln/ln.indy"    
    wget.download(url,lnIndustryFileName)  

allLN =  pd.read_table(lnFileName, sep='\t', header=0,names=['series_id', 'year', 'period', 'value', 'footnote_codes'])

LNSeriesDF =  pd.read_table(lnSeriesFileName, sep='\t', header=0,
                            names=['series_id', 'lfst_code', 'periodicity_code', 'series_title','absn_code', 'activity_code', 'ages_code', 'cert_code', 'class_code',
                                   'duration_code', 'education_code', 'entr_code', 'expr_code',
                                   'hheader_code', 'hour_code', 'indy_code', 'jdes_code', 'look_code',
                                   'mari_code', 'mjhs_code', 'occupation_code', 'orig_code', 'pcts_code',
                                   'race_code', 'rjnw_code', 'rnlf_code', 'rwns_code', 'seek_code',
                                   'sexs_code', 'tdat_code', 'vets_code', 'wkst_code', 'born_code',
                                   'chld_code', 'disa_code', 'seasonal', 'footnote_codes', 'begin_year',
                                   'begin_period', 'end_year', 'end_period'],
                            dtype = {'indy_code':np.str_,  'ages_code':np.str_,'education_code':np.str_,'race_code':np.str_}) 





e:\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



In [4]:
industriesDF =  pd.read_table(lnIndustryFileName, sep='\t', header=0,dtype={'indy_code':np.str_})    

In [5]:
# Only interested in 2020
# CES Seasonally Adjusted
condition = allLN['year'].isin([2020])  & allLN['period'].str.startswith('M')
LNS2020DF = allLN[condition]
LNS2020DF.shape

(59611, 5)

In [6]:
# Get month from period and dop period
# month 13 Annual Average      
# https://download.bls.gov/pub/time.series/ce/ce.period
LNS2020DF['Month'] = LNS2020DF['period'].str.replace('M','').astype(int)
LNS2020DF.drop('period',1,inplace=True)
LNS2020DF.shape

e:\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(59611, 5)

In [7]:
# Merge with Series DF
LNS2020DF = pd.merge(LNS2020DF,LNSeriesDF,on='series_id',how='left')
LNS2020DF.shape


(59611, 45)

In [8]:
LNS2020DF = pd.merge(LNS2020DF,industriesDF,on='indy_code',how='left')
LNS2020DF.shape

(59611, 46)

In [9]:
LNS2020DF.drop(['jdes_code','absn_code','cert_code','rjnw_code', 'begin_year',
                'begin_period', 'end_year', 'end_period'],1,inplace=True)



In [10]:
# Browse the data
# uncomment the following lines to browse the data

#d = dtale.show(LNS2020DF, ignore_duplicate=True)
#d.open_browser()

In [11]:
LNS2020DF.to_pickle("resources/LNS2020.pkl",compression='gzip')

In [12]:
# to load Picke file use the following once pickle has been generated
LNS2020DF_2=pd.read_pickle('resources/LNS2020.pkl',compression='gzip')